In [ ]:
import pandas as pd
import requests # библиотека предназначена для работы с запросами к серверу
from bs4 import BeautifulSoup # для раскрывания и работы с HTML файлом
import requests

In [2]:
def fetch_html(url): # функция которая забирает HTML c сайта
    try:
        response = requests.get(url) # берет и получает ответ от сервера
        response.raise_for_status()  # Check if the request was successful печатает статус 
        return response.text  # Return the HTML content of the page возврощает контент HTML
    except requests.exceptions.RequestException as e:  # логика которая выполняется если произойдет ошибка
        print(f"An error occurred: {e}")
        return None

In [3]:
URL = "https://www.sravni.ru/vklady/top/?sortProperty=rate&conditions=haveCapitalization"
response = fetch_html(URL)
html = BeautifulSoup(response, 'html.parser') 
#BeautifulSoup - cервер отчечает всегда HTML файлом, преобразует HTML из строки в свой формат, 
#это позволяет обращаться к элементам гараздо эффективнее используя имена блоков, ссылки, по тегам работать.

In [4]:
banks_data = html.find_all(class_="style_wrapper__c9ahs style_commonMargin__GJPlL") #find_all - возврощаем все блоки с классом
banks_data

[<div class="style_wrapper__c9ahs style_commonMargin__GJPlL"><div class="Card_wrapper__0WEM7"><div class="lazyload-wrapper"><div class="lazyload-placeholder"></div></div><div class="Card_card__kBcKQ Card_advertisingCard__FQP87"><div class="_1l6wwe5" style='--grid-template-areas:"logo licenseGroup firstCellGroup secondCellGroup thirdCellGroup detailsToggle button"
           "logo ratingGroup firstCellGroup secondCellGroup thirdCellGroup detailsToggle button";--grid-template-columns:44px minmax(160px, 2fr) minmax(140px, 180px) minmax(140px, 180px) minmax(140px, 180px) 46px 178px;--grid-template-rows:repeat(2, max-content);gap:0 16px'><div class="_15zg3r5" style="width:44px;height:44px;grid-area:logo"><div class="_qxd09l"><img alt="Транскапиталбанк" class="_1d6omtb" loading="lazy" src="https://s91588.cdn.ngenix.net/organizations/logotypes/square/5bb4f767245bc22a520a60cd.svg" title="Транскапиталбанк"/></div></div><div class="_1l6wwe5" style='--grid-template-areas:"caption license";--grid-

In [5]:
for bank_data_ in banks_data:     # bank_data - переменная в цикле, которая пробигается в списке div
    try:
        bank_name = bank_data_.find(class_='_jp36ng').text
        interest_rate = bank_data_.find_all(class_='_hxz9o6 _18hwa2n')[0].text
        period_from = bank_data_.find_all(class_='_hxz9o6 _18hwa2n')[1].text
        loan_sum = bank_data_.find_all(class_='_hxz9o6 _18hwa2n')[2].text.replace('\xa0', '')
        print('\n', bank_name, '|' , interest_rate, '|', period_from, '|',loan_sum)
    except:
        print('Ошибка парсинга по классу')


 Транскапиталбанк | 22.5 % | 367 дней | 100000 – 3000000 ₽

 Московский Кредитный Банк | до 24 % | 95 – 1100 дней | 10000 – 3000000 ₽

 Альфа-Банк | до 22.02 % | 92 – 1095 дней | от 50000 ₽

 Т-Банк | до 24.13 % | 2 – 24 месяцев | от 50000 ₽

 МТС Банк | до 22.55 % | 91 – 1096 дней | от 10000 ₽

 Яндекс Банк | до 22 % | 91 – 730 дней | 10000 – 10000000 ₽

 Россельхозбанк | до 23 %до 21 % | 91 – 1095 дней | от 5000 ₽

 Газпромбанк | до 22.02 % | 181 день | от 15000 ₽

 БыстроБанк | 23 % | 181 – 191 дня | от 10000 ₽

 Примсоцбанк | до 23 %до 22.9 % | 212 дней | от 10000 ₽

 Интерпрогрессбанк | до 22.01 % | 367 дней | от 100000 ₽

 Альфа-Банк | до 20 % | от 1 дня | 1 – 1500000 ₽

 Газпромбанк | до 19 % | от 1 дня | 1 – 1500000 ₽

 Банк ДОМ.РФ | до 22.9 %до 22.6 % | 91 – 1100 дней | от 30000 ₽


### Метод POST

In [6]:
url = 'https://www.sravni.ru/proxy-deposits/products'

data = {
        "productName": "vklady",
        "limit": 300,
        "offset": 0,
        "location": "6.83.",
        "isMix": False,
        "advertisingOnly": False,
        "filters": {
            "group": [
                "organization"
            ],
            "groupLimit": 5,
            "sortProperty": "popularity",
            "sortDirection": "desc",
            "adsAll": True
        }
    }

response = requests.post(url, json=data)
data = response.json()

# Нашел url , который позволяет 

In [7]:
len(data["items"])

187

In [8]:
for item in data["items"]:
    print('\nБанк: ', item["name"])
    print('Ставка: ', item["rate"])
    print('Срок: ', item["term"])
    print(f'Сумма от {item["amount"]["from"]} до {item["amount"]["to"]}')
    print('\nЧисло Дополнительныйх Предложений: ', item["groupCount"])
    if item["groupCount"] > 0:
        for i, group_item in enumerate(item['groupItems'], start=1):  #enumerate - каждому объекту задает индекс
            print(f'\nПредложение {i}')
            print('Ставка: ', group_item["rate"])
            print('Срок: ', group_item["term"])
            print(f'Сумма от {group_item["amount"]["from"]} до {group_item["amount"]["to"]}')


Банк:  ТКБ. Высокий доход_367
Ставка:  22.5 %
Срок:  367 дней
Сумма от 100000 до 3000000

Число Дополнительныйх Предложений:  3

Предложение 1
Ставка:  22.5 %
Срок:  181 день
Сумма от 100000 до 3000000

Предложение 2
Ставка:  21.1 %
Срок:  91 день
Сумма от 100000 до 3000000

Предложение 3
Ставка:  10.25 %
Срок:  1110 дней
Сумма от 100000 до 3000000

Банк:  МКБ. Перспектива (% в конце срока)
Ставка:  до 24 %
Срок:  95 – 1100 дней
Сумма от 10000 до 3000000

Число Дополнительныйх Предложений:  0

Банк:  Альфа-Вклад. Максимальный
Ставка:  до 21.05 %
Срок:  92 – 1095 дней
Сумма от 50000 до None

Число Дополнительныйх Предложений:  3

Предложение 1
Ставка:  до 19.22 %
Срок:  92 – 1095 дней
Сумма от 10000 до None

Предложение 2
Ставка:  до 16.09 %
Срок:  92 – 365 дней
Сумма от 300000 до 200000000

Предложение 3
Ставка:  до 15.99 %
Срок:  92 – 365 дней
Сумма от 300000 до 200000000

Банк:  Накопительный
Ставка:  до 23 %
Срок:  от 1 дня
Сумма от 1 до 1500000

Число Дополнительныйх Предложений: 

In [9]:
# План:
# 1. Создать две таблицы: 1 таблица - банки, 2 таблица - предложения банка;
# 2. Определить связь междку таблицами;
# 3. Таблицы храним в pandase

In [10]:
def get_online_rate(item):
    """
    Функция получающая онлайн ставку для банка (если имеется).
    Используем try/except для отлавливания ошибок. 
    Если у банка нет онлайн ставки поле "Онлайн", то записываем 0, иначе значение онлайн ставки.
    """
    try:
        values = item["rateTable"]["Онлайн"]["rows"][0]

        percentages = [
            float(percent.replace('*', '').strip('%'))
            for item in values
            for percent in item.split(' / ')
            if '%' in percent
        ]
        return max(percentages)
    
    # except блок срабатывает когда возникает ошибка в try. Exception - это любая ошибка которая может возникнуть в try
    except Exception:
        return 0

In [11]:
bank_dict = {
    "id": [],
    "name": [],
    "rate": [],
    "online_rate": [],
    "term": [],
    "amount_from": [],
    "amount_to": [],
    "offer_count": []
}

bank_offers_dict = {
    "bank_id": [],
    "bank_name": [],
    "rate": [],
    "online_rate": [],
    "term": [],
    "amount_from": [],
    "amount_to": [],
}

for i, item in enumerate(data["items"], start=1):
    bank_dict["id"].append(i)
    bank_dict["name"].append(item["organization"]["name"]["short"])
    bank_dict["rate"].append(item["rate"])

    # добавляем ставку онлайн банков
    bank_dict["online_rate"].append(
        get_online_rate(item)
    )

    bank_dict["term"].append(item["term"])
    bank_dict["amount_from"].append(item["amount"]["from"])
    bank_dict["amount_to"].append(item["amount"]["to"])
    bank_dict["offer_count"].append(item["groupCount"])

    if item["groupCount"] > 0:
        for group_item in item['groupItems']:  #enumerate - каждому объекту задает индекс
            bank_offers_dict["bank_id"].append(i)
            bank_offers_dict["bank_name"].append(item["organization"]["name"]["short"])
            bank_offers_dict["rate"].append(group_item["rate"])

            # добавляем ставку онлайн банков
            bank_offers_dict["online_rate"].append(
                get_online_rate(group_item)
            )

            bank_offers_dict["term"].append(group_item["term"])
            bank_offers_dict["amount_from"].append(group_item["amount"]["from"])
            bank_offers_dict["amount_to"].append(group_item["amount"]["to"])

### Поиск Наличия Полного Имени Банка

In [12]:
not_existing = [] # хранит несуществующие имен банков

for item in data["items"]:
    # проверяем есть ли короткое имя банка
    try:
        name = item["organization"]["name"]["short"]
    # если в try происходит ошибка -> срабатывает except и добавялем имя банка для которого нет короткого имени (not_existing)
    except:
        not_existing.append(item["name"])

# показывает сколько таких банков
len(not_existing)

0

### Фиксим Ставки для Онлайн Банков
Не все банки имеют онлайн формат. У онлайн банков есть параметр `Онлайн`

In [13]:
# data["items"][0]["rateTable"] # пример не онлайн банка
# data["items"][1]["rateTable"] # пример онлайн банка

In [28]:
row_data = data["items"][2]["rateTable"]["Онлайн"]["rows"][0]

percentages = [
    float(percent.replace('*', '').strip('%'))
    for item in row_data
    for percent in item.split(' / ')
    if '%' in percent
]

max_percentage = max(percentages)
max_percentage

22.02

In [29]:
# исходный список ставок
data["items"][2]["rateTable"]["Онлайн"]["rows"][0]

['от 50 000 ₽',
 '18.71% / 19.01%*',
 '20.47% / 21.01%*',
 '21.05% / 22%*',
 '20.54% / 22.02%*',
 '20.05% / 22%*',
 '17.23% / 19.51%*',
 '16.21% / 18.99%*',
 '16.4% / 20.99%*']

In [30]:
# предобработанный список ставок
percentages

[18.71,
 19.01,
 20.47,
 21.01,
 21.05,
 22.0,
 20.54,
 22.02,
 20.05,
 22.0,
 17.23,
 19.51,
 16.21,
 18.99,
 16.4,
 20.99]

In [32]:
# объяснение enumerate
lst = ['product A', 'product B', 'product C']
for i, value in enumerate(lst):
    print(i, value)

0 product A
1 product B
2 product C


In [16]:
# создаем таблицы для Банков и Предложений Банков
bank_df = pd.DataFrame(bank_dict)
offers_df = pd.DataFrame(bank_offers_dict)

# Отфильтровал только по банкам, имеющие онлайн ставки
online_bank_df = bank_df[bank_df['online_rate'] != 0]
online_offer_df = offers_df[offers_df['online_rate'] != 0]

In [17]:
online_bank_df.head()

,id,name,rate,online_rate,term,amount_from,amount_to,offer_count
1,2,Московский Кредитный Банк,до 24 %,24.00,95 – 1100 дней,10000,3000000.0,0
2,3,Альфа-Банк,до 21.05 %,22.02,92 – 1095 дней,50000,NaN,3
3,4,Газпромбанк,до 23 %,0.01,от 1 дня,1,1500000.0,0
4,5,Т-Банк,до 21 %,24.13,2 – 24 месяцев,50000,NaN,1
6,7,Яндекс Банк,до 22 %,22.00,91 – 730 дней,10000,10000000.0,0


In [18]:
bank_name = 'Россельхозбанк'
online_bank_df[online_bank_df['name'] == bank_name]

,id,name,rate,online_rate,term,amount_from,amount_to,offer_count
7,8,Россельхозбанк,до 23 %,21.0,91 – 1095 дней,5000,NaN,0
58,59,Россельхозбанк,до 23 %,21.0,180 – 1095 дней,10000,NaN,14


- Дальше будем сравнивать `rate` и `online_rate` и выбираем всегда наибольшую ставку, это может быть `online_rate` или `rate`

In [19]:
# bank
# добавим "final_rate", предварительно предобработаем "rate" -> убираем %
bank_df['rate'] = bank_df['rate'].apply(lambda x: x.strip('до% ')) # возвращает строку
bank_df['rate'] = bank_df['rate'].astype('float') # меняем тип данных на вещественный

# выбираем наибольшую ставку между "rate" и "online_rate"
bank_df['final_rate'] = bank_df[['rate', 'online_rate']].max(axis=1) # параметр axis=1 говорит что нужно найти самое большое значение по строчке

- apply позволяет применять преобразования или функции ко всему столбцу сразу, а не к 1 значению -> в этом эффективность функции
- lambda это функция которая применяется для каждого элемента столбца


In [20]:
# bank online
online_bank_df['rate'] = online_bank_df['rate'].apply(lambda x: x.strip(' до%'))
online_bank_df['rate'] = online_bank_df['rate'].astype('float')

# выбираем наибольшую ставку между "rate" и "online_rate"
online_bank_df['final_rate'] = online_bank_df[['rate', 'online_rate']].max(axis=1)
online_bank_df

c:\Users\vlad\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\Users\vlad\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\Users\vlad\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be se

,id,name,rate,online_rate,term,amount_from,amount_to,offer_count,final_rate
1,2,Московский Кредитный Банк,24.00,24.00,95 – 1100 дней,10000,3000000.0,0,24.00
2,3,Альфа-Банк,21.05,22.02,92 – 1095 дней,50000,NaN,3,22.02
3,4,Газпромбанк,23.00,0.01,от 1 дня,1,1500000.0,0,23.00
4,5,Т-Банк,21.00,24.13,2 – 24 месяцев,50000,NaN,1,24.13
6,7,Яндекс Банк,22.00,22.00,91 – 730 дней,10000,10000000.0,0,22.00
7,8,Россельхозбанк,23.00,21.00,91 – 1095 дней,5000,NaN,0,23.00
8,9,Газпромбанк,22.02,25.00,181 день,15000,NaN,0,25.00
10,11,Примсоцбанк,23.00,23.00,212 дней,10000,NaN,0,23.00
12,13,Альфа-Банк,20.00,20.00,от 1 дня,1,1500000.0,0,20.00
13,14,Газпромбанк,19.00,19.00,от 1 дня,1,1500000.0,0,19.00


In [21]:
# сохраняем в excel
bank_df.to_excel('banks.xlsx', index=False)
offers_df.to_excel('offers.xlsx', index=False)